# HyLoRADA Unified (v0.4.0)

**Parameter-efficient fine-tuning for long-context LLMs.**

New Features in v0.4.0:
- **LongLoRA**: Trainable Embeddings & Norms
- **SinkLoRA**: Sink Token Support
- **YaRN**: RoPE Scaling

In [ ]:
# Setup
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Clone repo
!git clone https://github.com/SadiaTabassum1216/hylorada.git
%cd hylorada

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate tqdm bitsandbytes peft

In [ ]:
# Check GPU
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Quick Start
Standard HyLoRADA configuration for general fine-tuning.

In [ ]:
from hylorada import HyLoRADAConfig, HyLoRADAModel
from transformers import AutoModelForCausalLM

# Load a small model for demonstration
try:
    base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B", trust_remote_code=True)
    
    # Standard Config (Efficiency)
    config = HyLoRADAConfig(lora_rank=8)
    model = HyLoRADAModel(base_model, config)
    
    print("\n=== Standard Config ===")
    model.print_trainable_params()
except Exception as e:
    print(f"Model loading skipped (Internet required): {e}")

## 2. Long-Context Configuration (>32k Tokens)
Enable advanced features for long-context learning:
- `train_embeddings=True`: Unlock embedding layer training (LongLoRA/LongQLoRA)
- `s2_sink_tokens=4`: Enable sink tokens for stable attention (SinkLoRA)
- `rope_scaling_type="linear"`: Apply positional interpolation (YaRN)


In [ ]:
# Long-Context Config
long_context_config = HyLoRADAConfig(
    lora_rank=8,
    # LongLoRA
    train_embeddings=True,  
    train_norms=True,
    # SinkLoRA
    s2_attn_enabled=True,
    s2_sink_tokens=4,
    # YaRN / LongRoPE
    rope_scaling_type="linear",
    rope_scaling_factor=2.0
)

try:
    model_long = HyLoRADAModel(base_model, long_context_config)
    
    print("\n=== Long-Context Config ===")
    model_long.print_trainable_params()
except NameError:
    pass # base_model not loaded

## 3. Run Benchmark

In [ ]:
# Run benchmark comparison
!python run_benchmark.py --methods lora hylorada --epochs 1 --num_train 200

In [ ]:
# Analyze results
import json, os, glob

for f in glob.glob('./**/benchmark*.json', recursive=True):
    print(f"\n=== {f} ===")
    try:
        data = json.load(open(f))
        if 'results' in data:
            for m, r in data['results'].items():
                metric = 'perplexity' if 'perplexity' in r else 'accuracy'
                val = r.get(metric, 'N/A')
                if isinstance(val, float):
                    val = f"{val:.2f}"
                print(f"{m}: {metric}={val}")
    except Exception as e:
        print(f"Error reading {f}: {e}")